<a href="https://colab.research.google.com/github/chrisrichardson/linear-algebra/blob/main/02_Direct_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the libraries we need
!pip install pyamg
import numpy as np
import pyamg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.2 MB/s eta 0:00:00


# Direct solvers

A direct solver takes the Matrix `A` and manipulates it to solve the problem `A.x = b`, in much the same way as we would solve simultaneous equations by hand. Gaussian elimination involves taking rows, multiplying and combining them to introduce zeros where we want them.

### Model problem
We will use `pyamg` to generate some matrices for us.

In [78]:
# This will create a 4x4 matrix
A = np.array(pyamg.gallery.laplacian.poisson((2, 2)).todense())
A

array([[ 4., -1., -1.,  0.],
       [-1.,  4.,  0., -1.],
       [-1.,  0.,  4., -1.],
       [ 0., -1., -1.,  4.]])

In [79]:
# Some example input data
y = np.array([1.0, 2.0, 3.0, 4.0])

In [80]:
# MatVec b = A.y
b = A @ y
b

array([-1.,  3.,  7., 11.])

In [81]:
# numpy has a built-in solver, so we should be able to solve A.x = b, and get back to our original input "y".
np.linalg.solve(A, b)

array([1., 2., 3., 4.])

In [85]:
# Let's try to solve it ourselves by Gaussian elimination

# First, let us make an "augmented" matrix, by glueing the RHS vector onto A:

B = np.concatenate((A, b.reshape(4, 1)), axis=1)
print(B)

# This represents the system of equations, and we can now manipulate it by
# adding and subtracting rows from each other



[[ 4. -1. -1.  0. -1.]
 [-1.  4.  0. -1.  3.]
 [-1.  0.  4. -1.  7.]
 [ 0. -1. -1.  4. 11.]]


In [86]:
# Now, we could get rid of B[1, 0] and B[2, 0] if we add row 0 divided by 4 to rows 1 and 2.
B[1, :] = B[1, :] + B[0, :]/4
B[2, :] = B[2, :] + B[0, :]/4
print(B)

[[ 4.   -1.   -1.    0.   -1.  ]
 [ 0.    3.75 -0.25 -1.    2.75]
 [ 0.   -0.25  3.75 -1.    6.75]
 [ 0.   -1.   -1.    4.   11.  ]]


In [87]:

# We could write a function to do this
def clear_down(B, row):
    # Get the diagonal entry on this row
    diag = B[row, row]
    # Clear all entries below the diagonal by subtracting scaled copies of this row
    for i in range(row + 1, B.shape[0]):
        B[i, :] -= B[row, :] * B[i, row]/diag
    return B

B0 = clear_down(B, 0)
B1 = clear_down(B0, 1)
B2 = clear_down(B1, 2)
print(B2)


[[ 4.         -1.         -1.          0.         -1.        ]
 [ 0.          3.75       -0.25       -1.          2.75      ]
 [ 0.          0.          3.73333333 -1.06666667  6.93333333]
 [ 0.          0.          0.          3.42857143 13.71428571]]


In [89]:
# The system can now easily be solved

# Extract the 4x4 matrix and 4x1 vector from B2
A = B2[:4, :4]
b = B2[:, 4]

x = np.empty(4)

# A[3, 3] * x[3] = b[3]
x[3] = b[3] / A[3, 3]

# A[2, 2] * x[2] + A[2, 3] * x[3] = b[2] - but we know x[3] now... so rearrange
x[2] = (b[2] - x[3] * A[2, 3]) / A[2, 2]

# etc
x[1] = (b[1] - x[2] * A[1, 2] - x[3] * A[1, 3]) / A[1, 1]
x[0] = (b[0] - x[1] * A[0, 1] - x[2] * A[0, 2] - x[3] * A[0, 3]) / A[0, 0]

# Should be [1. 2. 3. 4.]
print(x)

[1. 2. 3. 4.]


## Exercise

* Write a complete function which performs Gaussian elimination on a matrix + vector system

* Test your code on a larger matrix, e.g.
* `A = np.array(pyamg.gallery.laplacian.poisson((5, 5)).todense())`
* `A = np.random.rand(5, 5)`

In [ ]:
# Hint: use the clear_down() routine above as a starting point
# Full solution is below!





In [ ]:
#@title Solution to exercise

def gaussian_elim(A, b):
  B = np.concatenate((A, b.reshape(b.shape[0], 1)), axis=1)
  for row in range(B.shape[0] - 1):
    diag = B[row, row]
    for i in range(row + 1, B.shape[0]):
      B[i, :] -= B[row, :] * B[i, row]/diag

  x = np.empty(B.shape[0])
  for i in range(B.shape[0] - 1, -1, -1):
    x[i] = (B[i, -1] - np.dot(B[i, i + 1:-1], x[i + 1:])) / B[i, i]

  return x

A = np.random.rand(5, 5)
b = np.arange(A.shape[0])

print(A)
print(b)
y = A @ b
print(y)

np.set_printoptions(suppress=True)
print(gaussian_elim(A, y))
